In [3]:
####  Insertar Video en la imagen de la Webcam  #####



from keras.models import load_model
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import cv2
import imageio
import os

# Load the saved model
from keras.models import load_model
model = load_model('model1.h5')  # <-- modelo salvado 

#Carga imagenes png 
bigote_img = imageio.imread("images/bigote2x.png") # images/bigote2.png
mascara = imageio.imread("images/stormtrooper-ojos-transparentes.png") #stormtrooper-ojos-transparentes.png
lenterojo_img = imageio.imread("images/lenterojo.png")
sombrero_img = imageio.imread("images/sombrerox.png")
tapaboca_img = imageio.imread("images/tapaboca.png")
stormtroopere_img = imageio.imread("images/stormtrooper-ojos-transparentes.png")
# Get Image dimensions



x = 50
y = 50
pc = {
    'x':20, 'y':20,
    'w':50, 'h':50
} 
pa = {
    'x': 320, 'y': 20,
    'w': 50, 'h': 50
}

#Redimensiona la imagen
bigote_img_chico = cv2.resize(bigote_img, (200, 100 )) 
sombrero_img_chico = cv2.resize(sombrero_img, (150, 150)) 
lenterojo_img_chico = cv2.resize(lenterojo_img, (250, 100 )) 
mascara_chico = cv2.resize(mascara, (50, 50) )
stormtrooper_img_chico = cv2.resize(mascara, (300, 300) )


def get_points_main(img):

    def detect_points(face_img):
        me  = np.array(face_img)/255
        x_test = np.expand_dims(me, axis=0)
        x_test = np.expand_dims(x_test, axis=3)

        y_test = model.predict(x_test)
        label_points = (np.squeeze(y_test)*48)+48


        return label_points
    

    # load haarcascade
    face_cascade = cv2.CascadeClassifier('model2/haarcascade_frontalface_default.xml')
    dimensions = (96, 96)


    try:
        default_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        gray_img = cv2.cvtColor(default_img, cv2.COLOR_RGB2GRAY)
        faces = face_cascade.detectMultiScale(gray_img, 1.3, 5)
#         faces = face_cascade.detectMultiScale(gray_img, 4, 6)

    except:
        return []

    faces_img = np.copy(gray_img)

    plt.rcParams["axes.grid"] = False


    all_x_cords = []
    all_y_cords = []


    for i, (x,y,w,h) in enumerate(faces):

        h += 10
        w += 10
        x -= 5
        y -= 5

        try:
            just_face = cv2.resize(gray_img[y:y+h,x:x+w], dimensions)
        except:
            return []
        cv2.rectangle(faces_img,(x,y),(x+w,y+h),(255,0,0),1)

        scale_val_x = w/96
        scale_val_y = h/96

        label_point = detect_points(just_face)

        all_x_cords.append((label_point[::2]*scale_val_x)+x)
        all_y_cords.append((label_point[1::2]*scale_val_y)+y)



    final_points_list = []
    try:
        for ii in range(len(all_x_cords)):
            for a_x, a_y in zip(all_x_cords[ii], all_y_cords[ii]):
                final_points_list.append([a_x, a_y])
    except:
        return final_points_list

    return final_points_list

cap = cv2.VideoCapture(0)

ret, frame = cap.read()

height, width, channel = frame.shape


# Array para ubicar localizaciòn los keypoints  
tags = ['','Bigote','','Barba','','','','','','','Nariz','','Ojo1','','Ojo2']
ojo1 = 0
ojo2 = 0
nariz = 0
bigote = 0
barba = 0
sombrero = 0
storm = 0

# funcion para insertar imagen en video 
def agregar_imagen(fondo, imagen, x, y):
    # verificar si la imagen tiene informacion de opacidad
    alto = imagen.shape[0]
    ancho = imagen.shape[1]
    if imagen.shape[-1] == 4:
        # normalizar la opacidad
        opacidad = imagen[:,:,3]/255
        # alpha blending
        #print(f" imagen.shape[-1] {imagen.shape[-1]}")
        # generar una imagen vacia
        imagen_3_canales = np.zeros((imagen.shape[0], imagen.shape[1], 3))
        
        # a cada canal multiplicarle la opacidad
        imagen_3_canales[:,:,0] = imagen[:,:,0] * opacidad
        imagen_3_canales[:,:,1] = imagen[:,:,1] * opacidad
        imagen_3_canales[:,:,2] = imagen[:,:,2] * opacidad

        # a la imagen de fondo, se le suma la imagen con informacion de opacidad
        fondo[y:y+alto, x:x+ancho, :] = (1-np.stack([opacidad, opacidad, opacidad], axis=-1)) * fondo[y:y+alto, x:x+ancho, :] + imagen_3_canales
    else:
        # reemplazamos la informacion del fondo
        fondo[y:y+alto, x:x+ancho, :] = imagen
# funcion para inserar imagen en video
def agregar_imagen_a_imagen(img_a, img_b, x, y):
    # determinar si la imagen b tiene información de la opacidad
    #print(f"img_b.shape[-1] {img_b.shape[-1]} ")
    if img_b.shape[-1] == 4: # existe canal de opacidad
        #print(f"img_b.shape[-1] {img_b.shape[-1]} ")
        # pixeles que no son transparentes
        mask = (img_b != 0).all(axis=2)
        
        # la porcion de la imagen A que reemplazaria con o sin transparencia
        bg = img_a[y:y+img_b.shape[0], x:x+img_b.shape[1], :]
        
        # copiando pixeles no transparentes de B a A
        bg[mask] = img_b[mask, :3]  
        
        img_a[y:y+img_b.shape[0], x:x+img_b.shape[1], :] = bg
    else:
        #print(f"else img_b.shape[-1] {img_b.shape[-1]} ")
        img_a[y:y+img_b.shape[0], x:x+img_b.shape[1], :] = img_b

while(True):
  # la captura se llama frame 
  ret, frame = cap.read()
  frame = cv2.flip( frame, 1 ) # flip horizontal para que sea un espejo  
  points = get_points_main(frame)
  #agregar_imagen_a_imagen(frame, bigote_img_chico, pa['x'], pa['y'])

  try:
     overlay = frame  #.copy()
  except Exception as e:
       print(e)
       break
  i = 0
  for point in points:
      
      i= i+1 # o_line_points son los puntos en tuplas de keypoints en el loog
  if len(points) != 0:
       o_line_points = [[12,13], [13,11], [11,14], [14,12], [12,10], [11,10], [10,3], [12,5], [11,3], [10,5], [10,4], [10,2], [5,1], [1,4], [2,0]]#, [0,3], [5,9], [9,8], [8,4], [2,6], [6,7], [7,3]]
       # Numero de Rostros detectados 
       num_face = len(points)//15    
       
       for i in range(num_face):
           line_points = np.array(o_line_points) + (15*(i))
           
           the_color = (189, 195, 199)
           j = 0 
           for ii in line_points:
                
                # Ejemplo: Si ojo està en 1, y es la posicion del ojo, inserta la imaggen mediante la funcion
                # en la posiciòn adecuada.
                #cv2.line(overlay, tuple(points[ii[0]]), tuple(points[ii[1]]), the_color, thickness=1)
                
                #pa = {'x': int(round(float(str(points[ii[0]][0])))), 'y': int(round(float(str(points[ii[0]][1])))),'w': 50, 'h': 50}
                
                agregar_imagen(frame, mascara_chico, 100, 180)
                
                #nariz = 0 if nariz == 0 else cv2.putText(overlay, str(tags[10]), tuple(points[ii[0]]), 1, 1, (255, 255, 255))
                if (ojo1 == 1 and j == 12):cv2.putText(overlay, str(tags[j]), tuple(points[ii[0]]), 1, 1, (255, 255, 255))
                if (ojo1 == 1 and j == 12):pa = {'x': int(round(float(str(points[ii[0]][0])))) - 80, 'y': int(round(float(str(points[ii[0]][1])))) - 25,'w': 50, 'h': 50}
                if (ojo1 == 1 and j == 12):agregar_imagen_a_imagen(frame, lenterojo_img_chico, pa['x'], pa['y'])
                
                if (sombrero == 1 and j == 9):pa = {'x': int(round(float(str(points[ii[0]][0])))) - 50, 'y': int(round(float(str(points[ii[0]][1])))) - 200,'w': 250, 'h': 250}
                if (sombrero == 1 and j == 9):agregar_imagen_a_imagen(frame, sombrero_img_chico, pa['x'], pa['y'])
                
                #if (ojo2 == 1 and j == 14):cv2.putText(overlay, str(tags[j]), tuple(points[ii[0]]), 1, 1, (255, 255, 255))    
                if (storm == 1 and j == 14):pa = {'x': int(round(float(str(points[ii[0]][0])))) -10 , 'y': int(round(float(str(points[ii[0]][1])))) +10 ,'w': 50, 'h': 50}
                if (storm == 1 and j == 14):agregar_imagen(frame, stormtrooper_img_chico, pa['x'], pa['y'])
                
                
                if (nariz == 1 and j == 10):cv2.putText(overlay, str(tags[j]), tuple(points[ii[0]]), 1, 1, (255, 255, 255))
                #if (nariz == 1 and j == 10):pa = {'x': int(round(float(str(points[ii[0]][0])))) -10 , 'y': int(round(float(str(points[ii[0]][1])))) +10 ,'w': 50, 'h': 50}
                #if (nariz == 1 and j == 10):agregar_imagen(frame, bigote_img_chico, pa['x'], pa['y'])    
                if (bigote == 1 and j == 1):cv2.putText(overlay, str(tags[j]), tuple(points[ii[0]]), 1, 1, (255, 255, 255))
                #if (bigote == 1 and j == 1):cv2.imshow('frame',bigote_img_chico)
                #if (bigote == 1 and j == 1):frame[bigy:bigy+big_height , bigx:bigx+big_width] = big
                #if (bigote == 1 and j == 1): agregar_imagen_a_imagen(frame, bigote_img_chico, pa['x'], pa['y'])
                #print(f"pa['x'] {pa['x']} - pa['y'] {pa['y']} ")
                #frame[ tuple(points[ii[0]]), tuple(points[ii[1]]) ] = big
                #print(f"tuple(points[ii[0]]) {tuple(points[ii[0]])} - tuple(points[ii[1]]) {tuple(points[ii[1]])}")
                #print(f"[points[ii[0]][0]] {[points[ii[0]][0]]} - [points[ii[0]][1]] {[points[ii[0]][1]]}")
                
                #if (barba == 1 and j == 3):agregar_imagen_a_imagen(frame, sombrero_img_chico, pa['x'], pa['y'])
                if (barba == 1 and j == 3):cv2.putText(overlay, str(tags[j]), tuple(points[ii[0]]), 1, 1, (255, 255, 255))    
                cv2.putText(overlay, str(f"Texto), (10,50), 1, 1, (255, 255, 255))
                cv2.putText(overlay, str(f"s = Sombrero"), (10,70), 1, 1, (255, 255, 255))
                cv2.putText(overlay, str(f"o = Anteojos"), (10,90), 1, 1, (255, 255, 255))
                cv2.putText(overlay, str(f"b = Bigote (Da error)"), (10,110), 1, 1, (255, 255, 255))
                cv2.putText(overlay, str(f"r = Barba (Da error)"), (10,130), 1, 1, (255, 255, 255))
                cv2.putText(overlay, str(f"t = Máscara (Da error)"), (10,150), 1, 1, (255, 255, 255))
                cv2.putText(overlay, str(f"n = Nariz"), (10,60), 1, 1, (255, 255, 255))
                #cv2.putText(overlay, str(tags[j]), tuple(points[ii[0]]), 1, 1, (255, 255, 255))
                #cv2.putText(overlay, str(j), tuple(points[ii[0]]), 1, 1, (255, 255, 255))
                the_color = (189, 195, 199)
                #cv2.circle(frame, (tuple(points[ii[0]])), 5, (0, 255, 0), -1)
                #cv2.circle(frame, (tuple(points[ii[1]])), 5, (255, 0, 0), -1)
                #print(f"Indice J es, {j},tuple(points[ii[0]])=,{tuple(points[ii[0]])},tuple(points[ii[1]]),{tuple(points[ii[1]])}")
                #print(f"{j}, {ii},{num_face},{tuple(points[ii[0]])},{tuple(points[ii[1]])},{tuple(point)} , ")
                #cv2.putText(overlay, str(tags[j]), (tuple(points[ii])), 1, 1, (255, 255, 255))
                #cv2.putText(overlay, str(point[0] ), tuple(point), 1, 1, (255, 255, 255))
                #cv2.circle(frame, (tuple(point)), 1, (0, 255, 0), -1)
                #cv2.circle(frame, (tuple(point)), 1, (0, 255, 0), -1)
                #j = j = 14 ? 0 : j + 1
                j = 0 if j == 14 else j+1
                
                k = cv2.waitKey(1)
                
                if k == ord('q'):
                    #print("Pressed q")
                    break
                    
                elif k == ord('n'):
                    nariz = 0 if nariz == 1 else 1
                    print(f"Pressed n nariz = {nariz}")
                elif k== ord('o'):
                    ojo1 = 0 if ojo1 == 1 else 1
                    ojo2 = 0 if ojo2 == 1 else 1
                    print(f"Pressed o ojo1 = {ojo1}")
                   
      
                elif k== ord('b'):
                    bigote = 0 if bigote == 1 else 1
                    print(f"Pressed b bigote = {bigote}")
                elif k== ord('s'):
                    sombrero = 0 if sombrero == 1 else 1
                    print(f"Pressed s sombrero = {sombrero}") 
                elif k== ord('t'):
                    storm = 0 if storm == 1 else 1
                    print(f"Pressed t storm = {storm}")     
                elif k== ord('r'):
                    barba = 0 if barba == 1 else 1     
                    print(f"Pressed r barba = {barba}")
                
           

  opacity = 0.3
  cv2.addWeighted(overlay, opacity, frame, 1 - opacity, 0, frame)
        
  #out.write(frame)
  if k == ord('q'):
      #print("sale por q")
      break
  else:      
      cv2.imshow('frame',frame)

cap.release()
cv2.destroyAllWindows()


Pressed s sombrero = 1
Pressed s sombrero = 0
Pressed s sombrero = 1


In [2]:
cap.release()
cv2.destroyAllWindows()